# Proyek Analisis Data: [Input Nama Dataset]
- **Nama:** Firda Humaira
- **Email:** firdahumaira13@gmail.com
- **ID Dicoding:** firdahum

## Menentukan Pertanyaan Bisnis

- Apakah hari libur memiliki dampak signifikan terhadap jumlah penyewaan?
- Seberapa besar dampak suhu terhadap keputusan pengguna untuk menggunakan sepeda?
- Bagaimana pola penyewaan sepeda berdasarkan waktu dalam sehari (jam sibuk vs. jam sepi)?

## Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

: 

## Data Wrangling

### Gathering Data

In [ ]:
hour_df = pd.read_csv("hour.csv")
hour_df.head()

In [ ]:
day_df = pd.read_csv("day.csv")
day_df.head()

**Insight:**
- day.csv = Berisikan data yang sudah dirangkum, sehingga tidak mencatat detail penggunaan per jam dan tidak memiliki kolom hr.
- hour.csv = Berisikan data yang yang mencatat penyewaan tiap jam.

## Assessing Data

###Menilai Tabel


In [ ]:
hour_df.info()
day_df.info()

###Periksa data missing value

In [ ]:
hour_df.isna().sum()

In [ ]:
day_df.isna().sum()

###Memeriksa data yang duplikat

In [ ]:
print("Jumlah duplikasi hour_df: ", hour_df.duplicated().sum())
print("Jumlah duplikasi day_df: ", day_df.duplicated().sum())

Menampilkan ringkasan

In [ ]:
hour_df.describe()

In [ ]:
day_df.describe()

**Insight:**
- Terdapat kesalahan pada tipe data pada file hour_df dan day_df, pada variabel dteday seharusnya tipe datanya datetime
- Tidak terdapat data yang missing values
- Tidak ada data yang duplicate

## Cleaning Data

###Mengganti Tipe Data pada Variabel dteday

In [ ]:
datetime_columns = ["dteday"]

for column in datetime_columns:
  hour_df[column] = pd.to_datetime(hour_df[column])

In [ ]:
hour_df.info()

In [ ]:
datetime_columns = ["dteday"]

for column in datetime_columns:
  day_df[column] = pd.to_datetime(day_df[column])

In [ ]:
day_df.info()

**Insight:**
- Memperbaiki tipe data variabel dteday dari object ke datetime


## Exploratory Data Analysis (EDA)

### Explore day_df

####Rata-rata penyewaan per-bulan

In [ ]:
perbulan = day_df.groupby("mnth")["cnt"].mean()
perbulan

####Rata-rata Penyewaan Sepeda per Musim

In [ ]:
permusim = day_df.groupby("season")["cnt"].mean()
permusim

###Explore hour_df

In [ ]:
perjam = hour_df.groupby("hr")["cnt"].mean()
perjam

In [ ]:
casual_terdaftar = day_df.groupby("season")["casual"].sum()
casual_terdaftar

In [ ]:
merge_df = hour_df.merge(day_df[['dteday', 'cnt']], on='dteday', how='left', suffixes=('_hourly', '_daily'))
merge_df.head()

In [ ]:
merge_df.info()

**Insight:**
- day_df

**Rata-Rata Penyewaan Sepeda per Bulan**

Bulan dengan penyewaan tertinggi terjadi pada Juli dan Agustus, sementara bulan dengan penyewaan terendah cenderung berada di Januari dan Februari akibat musim dingin.


**Rata-Rata Penyewaan Sepeda per Musim**

Musim panas mencatatkan rata-rata penyewaan tertinggi. Musim gugur juga memiliki angka penyewaan yang cukup tinggi. Sebaliknya, musim dingin mencatatkan jumlah penyewaan terendah. Musim semi ada peningkatan jumlah penyewaan dibanding musim dingin, tetapi tetap lebih rendah dibandingkan musim panas.

- hour_df

**Rata-Rata Penyewaan Sepeda per Jam**

Penyewaan sepeda paling ramai di jam 07:00-09:00 dan 17:00-19:00. Pada siang hari (10:00-15:00), jumlah penyewaan tetap stabil namun lebih rendah dibandingkan jam sibuk. Sementara itu, pada dini hari (00:00-05:00), penyewaan sepeda berada pada titik terendah.

**Perbedaan Penyewaan antara Pengguna Kasual dan Terdaftar**

Pengguna terdaftar lebih sering nyewa sepeda di jam-jam sibuk. Sebaliknya, pengguna kasual lebih sering pakai sepeda pas akhir pekan atau di luar jam sibuk

## Visualization & Explanatory Analysis

- Apakah hari libur memiliki dampak signifikan terhadap jumlah penyewaan?
- Seberapa besar dampak suhu terhadap keputusan pengguna untuk menggunakan sepeda?
- Bagaimana pola penyewaan sepeda berdasarkan waktu dalam sehari (jam sibuk vs. jam sepi)?

### Pertanyaan 1: Apakah hari libur memiliki dampak signifikan terhadap jumlah penyewaan?

In [ ]:
# Cek jumlah hari libur dan hari biasa
jumlah_hari_libur = hour_df["holiday"].value_counts()
print("Jumlah hari libur:", jumlah_hari_libur.get(1, 0))
print("Jumlah hari biasa:", jumlah_hari_libur.get(0, 0))

In [ ]:
# Cek jumlah hari libur dan hari biasa
jumlah_hari_libur = day_df["holiday"].value_counts()
print("Jumlah hari libur:", jumlah_hari_libur.get(1, 0))
print("Jumlah hari biasa:", jumlah_hari_libur.get(0, 0))

In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(x=day_df["holiday"].map({0: "Hari Libur", 1: "Hari Biasa"}), y=merge_df["cnt_daily"], hue=merge_df["holiday"].map({0: "Hari Libur", 1: "Hari Biasa"}), palette="Set2", legend=False)
plt.xlabel("Jenis Hari")
plt.ylabel("Rata-rata Penyewaan")
plt.grid(True)
plt.show()

### Pertanyaan 2: Seberapa besar dampak suhu terhadap keputusan pengguna untuk menggunakan sepeda?

In [ ]:
merge_daily = merge_df.groupby("dteday").agg({"temp": "mean", "cnt_daily": "mean"}).reset_index()

plt.figure(figsize=(8, 5))
sns.regplot(x=merge_daily["temp"], y=merge_daily["cnt_daily"], scatter_kws={"alpha": 0.5}, line_kws={"color": "red"})
plt.title("Dampak Suhu terhadap Penyewaan Sepeda")
plt.xlabel("Suhu")
plt.ylabel("Jumlah Penyewaan Harian")
plt.grid(True)
plt.show()

### Pertanyaan 3: Bagaimana pola penyewaan sepeda berdasarkan waktu dalam sehari (jam sibuk vs. jam sepi)?

In [ ]:
# Define rush hours (morning: 7-9, evening: 17-19) and off-peak hours (0-5)
rush_hours = [7, 8, 9, 17, 18, 19]
off_peak_hours = [0, 1, 2, 3, 4, 5]

In [ ]:
# Separate data
rush_hour_rentals = perjam.loc[rush_hours]
off_peak_rentals = perjam.loc[off_peak_hours]

In [ ]:
plt.figure(figsize=(10, 5))

# Garis utama untuk total penyewaan sepeda
sns.lineplot(x=perjam.index, y=perjam.values, marker="o", label="Total Rentals", color="darkblue", linewidth=2)

# Titik merah untuk jam sibuk (rush hours)
sns.scatterplot(x=rush_hour_rentals.index, y=rush_hour_rentals.values, color="red", s=100, edgecolor="black", label="Rush Hours")

# Titik hijau untuk jam sepi (off-peak hours)
sns.scatterplot(x=off_peak_rentals.index, y=off_peak_rentals.values, color="green", s=100, edgecolor="black", label="Off-Peak Hours")

plt.title("Pola Penyewaan Sepeda Berdasarkan Waktu dalam Sehari", fontsize=14, fontweight="bold")
plt.xlabel("Jam", fontsize=12)
plt.ylabel("Rata-rata Penyewaan", fontsize=12)
plt.xticks(range(0, 24))
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend()
plt.show()


**Insight:**
- Penyewaan cenderung lebih tinggi pada hari biasa, sehingga bisnis dapat menerapkan strategi seperti promo di hari libur untuk meningkatkan permintaan dan menjaga tingkat penyewaan pada hari libur.
-Semakin tinggi suhu, semakin banyak penyewaan sepeda. Hal ini dapat dimanfaatkan dengan menambah stok sepeda dan layanan pada musim panas untuk menghadapi lonjakan permintaan.
- Penyewaan memuncak pada jam sibuk pagi dan sore hari, menunjukkan bahwa banyak pelanggan menggunakan sepeda untuk keperluan transportasi. Pada bisnis penyewaan ini dapat mengoptimalkan operasional, seperti meningkatkan ketersediaan sepeda dan layanan pada jam-jam tersebut untuk memberikan pengalaman terbaik bagi pelanggan.

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1 : Rata-rata penyewaan pada hari biasa lebih tinggi dibandingkan hari libur, seperti terlihat dari perbedaan tinggi batang pada diagram. Namun, terdapat variasi dalam data yang ditunjukkan oleh garis error di atas batang, menandakan fluktuasi jumlah penyewaan di kedua kategori.

- Conclution pertanyaan 2 : Terlihat adanya korelasi positif antara suhu dan jumlah penyewaan sepeda. Garis regresi merah menunjukkan bahwa semakin tinggi suhu, semakin banyak jumlah penyewaan sepeda. Namun, data juga menunjukkan adanya penyebaran yang cukup luas, terutama pada suhu tinggi, yang menandakan variasi dalam jumlah penyewaan meskipun suhu meningkat.

- Conclution pertanyaan 3 : Terlihat bahwa jumlah penyewaan meningkat tajam pada jam sibuk (rush hours), yaitu sekitar pukul 7-9 pagi dan 17-19 sore, yang ditandai dengan titik merah. Pola ini menunjukan bahwa sepeda banyak digunakan untuk keperluan perjalanan kerja atau sekolah. Sementara itu, jumlah penyewaan lebih rendah pada dini hari dan malam, dengan titik hijau menandai jam sepi (off-peak hours).